Import : First the standard library modules (e.g., os, gc, json), then data analysis libraries (numpy, pandas, matplotlib, PIL), followed by PyTorch/Torchvision imports (torch, nn, optim, torchvision, etc.), and finally additional packages like optuna, wandb, and sklearn


In [1]:
import os
import gc
import json
import unidecode
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset

import torchvision
from torchvision import datasets, models, transforms
from torchvision.models import VGG19_Weights

import optuna
import wandb
from sklearn.model_selection import KFold

import import_ipynb

######## 

from DATA_AND_MODELS import VanGoghDataset,VanGoghModel

wandb.login(key='ba8ed449ca151ad3f490026aec87d75b7171a16d')


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/gabriel/.netrc
wandb: Currently logged in as: gabi-dadashev (gabi-dadashev-tel-aviv-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

Load the CSV, keep selected columns, filter only **Post-Impressionism** images, add a 'is_van_gogh' flag, and reset the index.

In [2]:
classes = pd.read_csv("classes.csv", encoding="utf-8")
classes=classes[['filename', 'artist', 'genre', 'description', 'phash', 'width','height', 'genre_count']].copy()
classes_only_post_impressionism = classes[classes["filename"].str.contains('Post_Impressionism', case=False, na=False)].copy()
classes_only_post_impressionism['is_van_gogh'] = np.where(classes_only_post_impressionism['artist'] == 'vincent van gogh', 1, 0)
classes_only_post_impressionism=classes_only_post_impressionism.reset_index(drop=True)

Free unused GPU cache with PyTorch and invoke Python's garbage collector to release unreferenced objects

In [3]:
torch.cuda.empty_cache()
gc.collect()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

device

device(type='cuda')

In [4]:
dataset = VanGoghDataset(dataframe=classes_only_post_impressionism)
model_VGG19 = VanGoghModel(device,None,'VGG19')
model_Alex = VanGoghModel(device,None,'AlexNet')


In [5]:
def early_stop_check(patience, best_val_loss, best_val_loss_epoch, current_val_loss, current_val_loss_epoch):
    early_stop_flag = False  
    if current_val_loss < best_val_loss:
        best_val_loss = current_val_loss
        best_val_loss_epoch = current_val_loss_epoch
    else:
        if current_val_loss_epoch - best_val_loss_epoch > patience:
            early_stop_flag = True  
    return best_val_loss, best_val_loss_epoch, early_stop_flag


In [6]:
def train_model_with_hyperparams(model, train_loader, val_loader, optimizer, criterion, epochs, patience, trial):

    print('train_model_with_hyperparams')
    
    
    best_val_loss = float('inf')  
    best_val_loss_epoch = 0  
    early_stop_flag = False
    best_model_state = None  

    ### Epoch Loop iterate the all data observation X time
    
    for epoch in range(1, epochs + 1):
        model.train() 
        train_loss = 0.0 
        total_train_samples = 0 
        correct_train_predictions = 0 
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad() 
            
           
            
            outputs = model(inputs).squeeze(1)  
            loss = criterion(outputs.view(-1), labels)

            loss.backward() 
            optimizer.step()  

            train_loss += loss.item() * inputs.size(0)
            total_train_samples += inputs.size(0)

      
            predicted = (outputs > 0).float() 

            correct_train_predictions += (predicted == labels).sum().item()

        train_loss /= total_train_samples
        train_accuracy = correct_train_predictions / total_train_samples

        model.eval()  
        val_loss = 0.0 
        total_val_samples = 0 
        correct_val_predictions = 0

        ## Intenial-Batch Loop split the data to batches

        with torch.no_grad(): 
            for inputs, labels in val_loader: 
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
               
            
               
                loss = criterion(outputs.view(-1), labels)

                val_loss += loss.item() * inputs.size(0)
                total_val_samples += inputs.size(0)

                _, predicted = torch.max(outputs, 1)
                correct_val_predictions += (predicted == labels).sum().item()

        
        ##Validation 
        
        val_loss /= total_val_samples
        val_accuracy = correct_val_predictions / total_val_samples

        best_val_loss, best_val_loss_epoch, early_stop_flag = early_stop_check(patience, best_val_loss, best_val_loss_epoch, val_loss, epoch)

        ##Early Stopping
        
        if val_loss == best_val_loss:
            best_model_state = model.state_dict()
        ### Weights & Biases (W&B) decimantion 
        
        wandb.log({ 
            "Epoch": epoch,
            "Train Loss": train_loss,
            "Train Accuracy": train_accuracy,
            "Validation Loss": val_loss,
            "Validation Accuracy": val_accuracy
        })

        if early_stop_flag: 
            break 

        ### handle best model
        

    if best_model_state is not None: 
        torch.save(best_model_state, f"best_model_trial_{trial.number}.pt") 
   


    return best_val_loss

In [7]:

def objective(trial):

    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-4)
    batch_size = trial.suggest_int("batch_size", 4, 16, step=4)
    patience = 7
    k_folds = 5  
    
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    fold_losses = []   
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        
        print(f"--- Trial (Exp) {trial.number}, Fold {fold+1}/{k_folds} ---")


        train_dataset = Subset(dataset, train_idx)
        val_dataset = Subset(dataset, val_idx)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        model =  VanGoghModel(device,trial,None).model

        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        criterion = nn.BCEWithLogitsLoss()

        wandb.init( project="VanGogh-Classifier",
        config={
            "model": trial.params["model"],  
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "batch_size": batch_size,
            "k_folds": k_folds
        },
        name=f"trial_{trial.number}_{trial.params['model']}"
    )

        best_val_loss = train_model_with_hyperparams(
            model, train_loader, val_loader, optimizer, criterion, epochs=4, patience=patience, trial=trial
        )




        fold_losses.append(best_val_loss)

    avg_val_loss = sum(fold_losses) / len(fold_losses)
    print(f"Trial {trial.number}, Average Validation Loss across {k_folds} folds: {avg_val_loss:.4f}")


    ###Decumantation    

    model_type = trial.params["model"]
    global BEST_PARAMS, BEST_PARAMS_HISTORY
    if avg_val_loss < BEST_PARAMS[model_type]["loss"]:
        BEST_PARAMS[model_type]["loss"] = avg_val_loss
        BEST_PARAMS[model_type]["params"] = trial.params.copy()
        
        BEST_PARAMS_HISTORY[model_type].append({
            "trial": trial.number,
            "loss": avg_val_loss,
            "params": trial.params.copy()
        })
        
        with open(f"best_params_history_{model_type}.json", "w") as f:
            json.dump(BEST_PARAMS_HISTORY[model_type], f, indent=4)
        print(f"New best parameters for {model_type} saved: {trial.params}")




    
    del model 
    torch.cuda.empty_cache()
    
    return avg_val_loss

In [8]:
BEST_PARAMS = {
    "AlexNet": {"loss": float("inf"), "params": None},
    "VGG19": {"loss": float("inf"), "params": None}
}


BEST_PARAMS_HISTORY = {
    "AlexNet": [],
    "VGG19": []
}


In [9]:

study = optuna.create_study(direction="minimize") 
study.optimize(objective, n_trials=15)

[I 2025-02-18 11:47:06,289] A new study created in memory with name: no-name-11d97f53-6a9f-4595-8d33-6766a8c25252
/tmp/ipykernel_8926/2823819912.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
/tmp/ipykernel_8926/2823819912.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-4)


--- Trial (Exp) 0, Fold 1/5 ---


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


train_model_with_hyperparams
--- Trial (Exp) 0, Fold 2/5 ---
train_model_with_hyperparams
--- Trial (Exp) 0, Fold 3/5 ---
train_model_with_hyperparams
--- Trial (Exp) 0, Fold 4/5 ---
train_model_with_hyperparams
--- Trial (Exp) 0, Fold 5/5 ---
train_model_with_hyperparams


[I 2025-02-18 13:13:11,500] Trial 0 finished with value: 0.6931471116553085 and parameters: {'learning_rate': 5.313401180185412e-05, 'weight_decay': 1.116092803144322e-06, 'batch_size': 8, 'model': 'VGG19'}. Best is trial 0 with value: 0.6931471116553085.


...

Trial 12, Average Validation Loss across 5 folds: 0.6750
--- Trial (Exp) 13, Fold 1/5 ---
train_model_with_hyperparams
--- Trial (Exp) 13, Fold 2/5 ---
train_model_with_hyperparams
--- Trial (Exp) 13, Fold 3/5 ---
train_model_with_hyperparams
--- Trial (Exp) 13, Fold 4/5 ---
train_model_with_hyperparams
--- Trial (Exp) 13, Fold 5/5 ---
train_model_with_hyperparams


[I 2025-02-19 05:47:26,012] Trial 13 finished with value: 0.6712148422287679 and parameters: {'learning_rate': 1.3146815506531705e-05, 'weight_decay': 3.3494254041695756e-05, 'batch_size': 8, 'model': 'AlexNet'}. Best is trial 13 with value: 0.6712148422287679.


Trial 13, Average Validation Loss across 5 folds: 0.6712
New best parameters for AlexNet saved: {'learning_rate': 1.3146815506531705e-05, 'weight_decay': 3.3494254041695756e-05, 'batch_size': 8, 'model': 'AlexNet'}
--- Trial (Exp) 14, Fold 1/5 ---
train_model_with_hyperparams
--- Trial (Exp) 14, Fold 2/5 ---
train_model_with_hyperparams
--- Trial (Exp) 14, Fold 3/5 ---
train_model_with_hyperparams
--- Trial (Exp) 14, Fold 4/5 ---
train_model_with_hyperparams
--- Trial (Exp) 14, Fold 5/5 ---
train_model_with_hyperparams


[I 2025-02-19 07:02:12,085] Trial 14 finished with value: 0.6704581181874261 and parameters: {'learning_rate': 1.0369291948014812e-05, 'weight_decay': 8.292217788444539e-05, 'batch_size': 4, 'model': 'AlexNet'}. Best is trial 14 with value: 0.6704581181874261.


Trial 14, Average Validation Loss across 5 folds: 0.6705
New best parameters for AlexNet saved: {'learning_rate': 1.0369291948014812e-05, 'weight_decay': 8.292217788444539e-05, 'batch_size': 4, 'model': 'AlexNet'}
